In [ ]:
import cv2
import pandas as pd
from pathlib import Path

from google.colab import drive
drive.mount('/content/drive')

BASE_DIR = Path("/content/drive/Shareddrives/TissueMotionForecasting")
TRAIN_ROOT = BASE_DIR / "scared_data" / "train"

DATASETS_TO_USE = ["dataset_1", "dataset_2", "dataset_3"]
KEYFRAME_NAMES = [f"keyframe_{i}" for i in range(1, 6)]  # 1–5

rows = []

for ds_name in DATASETS_TO_USE:
    ds_root = TRAIN_ROOT / ds_name
    for kf_name in KEYFRAME_NAMES:
        kf_dir = ds_root / kf_name
        rgb_path = kf_dir / "data" / "rgb.mp4"
        if not rgb_path.exists():
            continue

        cap = cv2.VideoCapture(str(rgb_path))
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cap.release()

        if fps <= 0:
            continue

        delta_t_ms = 1000.0 / fps  # ms between frames

        rows.append({
            "dataset": ds_name,
            "keyframe": kf_name,
            "rgb_path": str(rgb_path),
            "fps": fps,
            "delta_t_ms": delta_t_ms,
            "num_frames": frame_count,
        })

df_fps = pd.DataFrame(rows)
pd.set_option("display.precision", 3)
print(df_fps)


Mounted at /content/drive
      dataset    keyframe                                           rgb_path  \
0   dataset_1  keyframe_1  /content/drive/Shareddrives/TissueMotionForeca...   
1   dataset_1  keyframe_2  /content/drive/Shareddrives/TissueMotionForeca...   
2   dataset_1  keyframe_3  /content/drive/Shareddrives/TissueMotionForeca...   
3   dataset_2  keyframe_1  /content/drive/Shareddrives/TissueMotionForeca...   
4   dataset_2  keyframe_2  /content/drive/Shareddrives/TissueMotionForeca...   
5   dataset_2  keyframe_3  /content/drive/Shareddrives/TissueMotionForeca...   
6   dataset_2  keyframe_4  /content/drive/Shareddrives/TissueMotionForeca...   
7   dataset_3  keyframe_1  /content/drive/Shareddrives/TissueMotionForeca...   
8   dataset_3  keyframe_2  /content/drive/Shareddrives/TissueMotionForeca...   
9   dataset_3  keyframe_3  /content/drive/Shareddrives/TissueMotionForeca...   
10  dataset_3  keyframe_4  /content/drive/Shareddrives/TissueMotionForeca...   

     fps  del

In [ ]:
import pandas as pd

ALL_HORIZONS = list(range(1, 21))  # n = 1..20

latency_rows = []
for _, row in df_fps.iterrows():
    ds = row["dataset"]
    kf = row["keyframe"]
    delta_t_ms = row["delta_t_ms"]

    for n in ALL_HORIZONS:
        eta_ms = n * delta_t_ms
        latency_rows.append({
            "dataset": ds,
            "keyframe": kf,
            "horizon_n": n,
            "delta_t_ms": delta_t_ms,
            "eta_ms": eta_ms,
        })

df_latency_all = pd.DataFrame(latency_rows)
pd.set_option("display.precision", 1)

example = df_latency_all.query("dataset == 'dataset_1' and keyframe == 'keyframe_1'")
print(example[["horizon_n", "eta_ms"]])


    horizon_n  eta_ms
0           1    40.0
1           2    80.0
2           3   120.0
3           4   160.0
4           5   200.0
5           6   240.0
6           7   280.0
7           8   320.0
8           9   360.0
9          10   400.0
10         11   440.0
11         12   480.0
12         13   520.0
13         14   560.0
14         15   600.0
15         16   640.0
16         17   680.0
17         18   720.0
18         19   760.0
19         20   800.0


The SCARED videos run at 25 fps, i.e., Δt ≈ 40 ms between frames. For a forecast horizon of n frames, this corresponds to a lead time of ηₙ = n·Δt. We computed this mapping for n ∈ [1,20], which spans ηₙ from 40 ms to 800 ms, and selected three representative horizons, n = 1, 5, 10, to cover short-, medium-, and long-horizon behavior while keeping the number of trained models manageable. Concretely, n = 1 (≈ 40 ms) probes almost frame-to-frame prediction with minimal latency, n = 5 (≈ 200 ms) represents an intermediate horizon where motion is clearly visible but still correlated with the current frame, and n = 10 (≈ 400 ms) serves as a challenging long-horizon setting for stress-testing the forecast models.

In [ ]:
df_summary = (
    df_latency_all
    .groupby("horizon_n")
    .agg(
        mean_delta_t_ms=("delta_t_ms", "mean"),
        mean_eta_ms=("eta_ms", "mean"),
        std_eta_ms=("eta_ms", "std"),
    )
    .reset_index()
)

print(df_summary)


    horizon_n  mean_delta_t_ms  mean_eta_ms  std_eta_ms
0           1             40.0         40.0         0.0
1           2             40.0         80.0         0.0
2           3             40.0        120.0         0.0
3           4             40.0        160.0         0.0
4           5             40.0        200.0         0.0
5           6             40.0        240.0         0.0
6           7             40.0        280.0         0.0
7           8             40.0        320.0         0.0
8           9             40.0        360.0         0.0
9          10             40.0        400.0         0.0
10         11             40.0        440.0         0.0
11         12             40.0        480.0         0.0
12         13             40.0        520.0         0.0
13         14             40.0        560.0         0.0
14         15             40.0        600.0         0.0
15         16             40.0        640.0         0.0
16         17             40.0        680.0     

To confirm that this mapping is consistent across all videos, we aggregated the frame-interval and latency statistics over all dataset/keyframe pairs (Table X). For each horizon $n \in [1,20]$, the mean frame interval remains $\Delta t \approx 40 \text{ ms}$ and the standard deviation of the induced latency $\eta_n$ is zero. This indicates that all RGB sequences in our subset of SCARED share the same frame rate (25 fps), so the mapping $\eta_n = n \cdot 40 \text{ ms}$ holds uniformly across the dataset.
